![MONAI Logo](monai.png)

# MONAI Deploy App SDK - BYOM Application

This tutorial you'll walk through creating your own MONAI Application Package (MAP) with a custom model.  We'll provide some boilerplate code and you'll do the rest!

### Table of Contents
* [Setup](#1.-Setup)
* [Create Operator Class](#2.-Create-Operator-Class)
* [Create Application Class](#3.-Create-Application-Class)
* [Excuting App Locally](#4.-Executing-app-locally)
* [Conclusion](#5.-Conclusion)

### Using Google Colab

This notebook has the pip command for installing MONAI and will be added to any subsequent notebook.

**Enabling GPU Support**

To use GPU resources through Colab, change the runtime to GPU:

1. From the **"Runtime"** menu select **"Change Runtime Type"**
2. Choose **"GPU"** from the drop-down menu
3. Click **"SAVE"**

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not)

### Verify GPU Access

Running **!nvidia-smi** in a cell will verify this has worked and show you what kind of hardware you have access to.    

In [ ]:
# Install necessary image loading/processing packages for the application
!pip install -q "Pillow"
!pip install -q "scikit-image"
!pip install -q "wget"
!pip install -q "pydicom"
!pip install -q "highdicom"
!pip install -q "matplotlib"
!pip install -q "typeguard==2.12.1"
%matplotlib inline

# Install MONAI Deploy App SDK package
!pip install -qU "monai-deploy-app-sdk"
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm, fire]==1.1.0"

### 1. Setup

To begin, check that the NVIDIA driver has been installed correctly. The `nvidia-smi` command should run and output information about the GPUs on your system:

In [ ]:
!nvidia-smi

### 1.1 Setup Environment

We'll set up folder called notebook_2 where we'll extract our data, models, and write out output.

In [ ]:
NOTEBOOK_ROOT="notebook_2/"
MODEL="pancreas_ct_dints_segmentation"
!mkdir -p {NOTEBOOK_ROOT}

### 1.2 Data Setup

Download/Extract ai_spleen_seg_data from Google Drive

In [ ]:
# Download ai_spleen_bundle_data test data zip file
!gdown "https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ"

# After downloading ai_spleen_bundle_data zip file from the web browser or using gdown,
!unzip -o "ai_spleen_seg_bundle_data.zip" -d "notebook_2"

### 1.3 Download Model

We're going to use the MONAI Bundle CLI to download a model from the [MONAI Model Zoo](https://monai.io/model-zoo).  For this notebook we're going to download the Pancreas CT DiNTS Segmentation Model and we're specifying version 0.3.6.

In [ ]:
!python -m monai.bundle download --name {MODEL}  --version "0.3.6" --bundle_dir {NOTEBOOK_ROOT}

### 1.4 Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [ ]:
import logging
from os import path

from numpy import uint8

import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    Invertd,
    LoadImaged,
    Orientationd,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
)

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.clara_viz_operator import ClaraVizOperator

## 2. Create Operator Class

You'll create the Pancreas Segmentation operator.  We've define the pre_process and post_process pipeline for you.

#### 2.1 Exercise
<div class="alert alert-block alert-info"> Implement compute() method </div>

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
class PancreasSegOperator(Operator):
    """Performs Pancreas and Pancreas Tumor segmentation with a 3D image converted from a DICOM CT series."""

    def __init__(self):
        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # This operator gets an in-memory Image object, so a specialized ImageReader is needed.
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # Delegates inference and saving output to the built-in operator.
        infer_operator = MonaiSegInferenceOperator(
            (
                96,
                96,
                96,
            ),
            pre_transforms,
            post_transforms,
        )

        # Setting the keys used in the dictironary based transforms may change.
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # Now let the built-in operator handles the work with the I/O spec and execution context.
        infer_operator.compute(op_input, op_output, context)


    def pre_process(self, img_reader) -> Compose:
        """Composes transforms for preprocessing input before predicting on a model."""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Orientationd(keys=my_key, axcodes="RAS"),
                Spacingd(keys=my_key, pixdim=[1.5, 1.5, 2.0], mode=["bilinear"]),
                ScaleIntensityRanged(keys=my_key, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
                EnsureTyped(keys=my_key)
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """Composes transforms for postprocessing the prediction results."""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                Invertd(keys=pred_key, transform=pre_transforms, orig_keys=self._input_dataset_key, nearest_interp=False),                
                AsDiscreted(keys=pred_key, argmax=True),
                SaveImaged(keys=pred_key, output_dir=out_dir, output_postfix="seg", output_dtype=uint8),
            ]
        )

## 3. Create Application Class

You'll implement the Application class for your MONAI Deploy Application.  Remember to use the MONAI Deploy provided DICOM operators and instantiate your Inference Operator class from above.  You'll then need to create a DAG by linking the operators together.

#### 3.1 Exercise
<div class="alert alert-block alert-info"> Implement compose() method </div>


In [ ]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
class AIPancreasTumorSegApp(Application):
    def __init__(self, *args, **kwargs):
        """Creates an application instance."""

        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        self._logger.debug(f"Begin {self.compose.__name__}")
        # Creates the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(rules=Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Model specific inference operator, supporting MONAI transforms.

        # Creates the model specific segmentation operator
        pancreas_seg_op = PancreasSegOperator()

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue.
        # The segment_label, algorithm_name, and algorithm_version are limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        # User can Look up SNOMED CT codes at, e.g.
        # https://bioportal.bioontology.org/ontologies/SNOMEDCT

        _algorithm_name = "3D segmentation of multi-Organs from a CT series"
        _algorithm_family = codes.DCM.ArtificialIntelligence
        _algorithm_version = "0.1.0"

        segment_descriptions = [
            SegmentDescription(
                segment_label="Multi",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Abdomen,                
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
        ]        
        
        dicom_seg_writer = DICOMSegmentationWriterOperator(segment_descriptions)

        # Create the processing pipeline, by specifying the source and destination operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, pancreas_seg_op, {"image": "image"})

        # Note below the dicom_seg_writer requires two inputs, each coming from a source operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(pancreas_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

        viz_op = ClaraVizOperator()
        self.add_flow(series_to_vol_op, viz_op, {"image": "image"})
        self.add_flow(pancreas_seg_op, viz_op, {"seg_image": "seg_image"})        
        
        self._logger.debug(f"End {self.compose.__name__}")

# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
# For list of string values, e.g. "ImageType": ["PRIMARY", "ORIGINAL"], it is a match if all elements
# are all in the multi-value attribute of the DICOM series.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)",
                "ImageType": ["PRIMARY", "ORIGINAL"]
            }
        }
    ]
}
"""     

## 4. Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the `dcm` and the Torch Script model at `model.ts`. Please use the actual path in your environment.

### 4.1 Exercise
<div class="alert alert-block alert-info"> You'll test your application locally before trying to package the application. We've provided the `Run` function and passed the correct parameters.  If you have different paths, please update before running.  All of the oeprators should pass and you should have a Clara Viz viewer open. </div>

In [ ]:
app = AIPancreasTumorSegApp()

app.run(input=NOTEBOOK_ROOT+"/dcm", output=NOTEBOOK_ROOT+"/output", model=NOTEBOOK_ROOT+MODEL+"/models/model.ts")

## 5. Conclusion

In this notebook, you have implemented a MONAI Deploy Application and utilized a pretrained model from the MONAI Model Zoo.

### 5.1 What's Next
You're now ready to use MONAI Deploy on any current or future model you're working on!  Check out the [MONAI Deploy Github](https://github.com/Project-MONAI/monai-deploy) to keep up to date on the latest changes.  And feel free to report an issue, submit a pull request, contribute in discussions, or attending the MONAI Deploy Working Group meeting.